In [1]:
import requests
from bs4 import BeautifulSoup

urls = ["https://store.steampowered.com/search/?filter=topsellers&tags=4191&supportedlang=english&ndl=1"]

#Action "https://store.steampowered.com/search/?tags=19&supportedlang=english&filter=topsellers&ndl=1" 
#SinglePlayer https://store.steampowered.com/search/?filter=topsellers&tags=4182&supportedlang=english&ndl=1
#Adventure https://store.steampowered.com/search/?filter=topsellers&tags=21&supportedlang=english&ndl=1
#Indie https://store.steampowered.com/search/?filter=topsellers&tags=492&supportedlang=english&ndl=1
#Simulation https://store.steampowered.com/search/?filter=topsellers&tags=599&supportedlang=english&ndl=1
#Strategy https://store.steampowered.com/search/?filter=topsellers&tags=9&supportedlang=english&ndl=1
#Casual https://store.steampowered.com/search/?filter=topsellers&tags=597&supportedlang=english&ndl=1
#RPG https://store.steampowered.com/search/?filter=topsellers&tags=122&supportedlang=english&ndl=1
#Multiplayer https://store.steampowered.com/search/?filter=topsellers&tags=3859&supportedlang=english&ndl=1
#Atmospheric https://store.steampowered.com/search/?filter=topsellers&tags=4166&supportedlang=english&ndl=1
#2d https://store.steampowered.com/search/?filter=topsellers&tags=3871&supportedlang=english&ndl=1 
#story rich https://store.steampowered.com/search/?filter=topsellers&tags=1742&supportedlang=english&ndl=1
#co-op https://store.steampowered.com/search/?filter=topsellers&tags=1685&supportedlang=english&ndl=1
#first-person https://store.steampowered.com/search/?filter=topsellers&tags=3839&supportedlang=english&ndl=1
#3d https://store.steampowered.com/search/?filter=topsellers&tags=4191&supportedlang=english&ndl=1
#open world https://store.steampowered.com/search/?filter=topsellers&tags=1695&supportedlang=english&ndl=1

app_id = []

for url in urls:
    response = requests.get(url)
# , headers={
#     'cookie': 'steamCountry=KR!13817a80e36b14f1a94dd77fb2142a64; browserid=3999848730290483524; sessionid=2225f1149bbcf81b9631038f; timezoneOffset=32400,0; app_impressions=2344520@1_7_7_7000_150_1|2138330@1_7_7_7000_150_1|2358720@1_7_7_7000_150_1|2074920@1_7_7_7000_150_1|2183900@1_7_7_7000_150_1|1245620@1_7_7_7000_150_1|2139460@1_7_7_7000_150_1|1049590@1_7_7_7000_150_1|1091500@1_7_7_7000_150_1|578080@1_7_7_7000_150_1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
#     'accept-language':'ko-KR,ko;q=0.9',
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
#     'accept-encoding': 'gzip, deflate, br, zstd',
#     'cache-control':'no-cache'
# }

    soup = BeautifulSoup(response.content, 'html.parser')

    app_ids = []

    container = soup.find('div', id='search_result_container')

    if container:
        for item in container.find_all('a'):
            link =item['href']
            if 'app' in link:
                ex = link.split('app')[1].split('/')[1]
                app_ids.append(ex)

    app_ids=app_ids[:10] # 상위 10개

    for i in range(len(app_ids)):
        app_id.append(app_ids[i])

app_id


['2139460',
 '1049590',
 '1245620',
 '2358720',
 '1364780',
 '2183900',
 '2322010',
 '2021880',
 '1462040',
 '872410']

In [2]:
import requests
import pandas as pd

# Steam Store API에서 특정 게임의 세부 정보를 가져오는 함수
def get_game_details(app_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data[str(app_id)]["success"]:
            game_data = data[str(app_id)]["data"]
            return {
                "app_id": app_id,
                "name": game_data.get("name"),
                "genres": ", ".join([genre['description'] for genre in game_data.get("genres", [])]),  # 장르
                "header_image": game_data.get("header_image"),  # 헤더 이미지
                "release_date": game_data.get("release_date", {}).get("date"),  # 출시일
                "price": game_data.get("price_overview", {}).get("final_formatted", "가격 정보 없음")  # 가격 정보
            }
    return {}

# Steam API를 통해 리뷰 데이터를 가져오는 함수
def get_reviews_for_game(app_id, num_reviews=100):
    url = f"https://store.steampowered.com/appreviews/{app_id}"
    params = {
        'json': 1,
        'num_per_page': num_reviews,  # 한 번에 가져올 리뷰 수
        'filter': 'recent',  # 최신 리뷰만 가져옴
        'language': 'english'  # 영어로 된 리뷰만 가져옴
    }
    response = requests.get(url, params=params)
    data = response.json()

    reviews = []

    if 'reviews' in data:
        for review in data['reviews']:
            reviews.append({
                'review_text': review.get('review'),  # 리뷰 내용
                'review_score': review.get('voted_up'),  # 추천 여부
                'language': review.get('language'),  # 리뷰 언어
                'author_steamid': review.get('author', {}).get('steamid'),  # 작성자 Steam ID
                'playtime_at_review': review.get('author', {}).get('playtime_at_review'),  # 리뷰 작성 당시의 플레이 시간
                'timestamp_created': review.get('timestamp_created')  # 리뷰 작성 시간
            })

    return reviews

# Steam ID를 사용자 닉네임으로 변환하는 함수 (API 키 사용하지 않음)
def resolve_vanity_url(steam_id):
    # API 키 없이, 닉네임을 직접 반환
    return steam_id  # 원래 Steam ID 반환

# 게임 이름, 장르, 이미지, 리뷰 정보 등을 포함한 데이터를 DataFrame으로 정리하는 함수
def get_game_data_with_reviews(app_ids, num_reviews=100):
    all_data = []

    for app_id in app_ids:
        # 1. 게임 세부 정보 가져오기
        game_info = get_game_details(app_id)
        
        if game_info:
            # 2. 게임에 대한 리뷰 가져오기
            reviews = get_reviews_for_game(app_id, num_reviews)
            
            # 3. 각 리뷰에 게임 세부 정보 추가
            for review in reviews:
                # 4. Steam ID를 사용자 닉네임으로 변환 (닉네임으로 처리됨)
                author_nickname = resolve_vanity_url(review['author_steamid'])
                
                all_data.append({
                    'app_id': game_info['app_id'],  # 게임 ID
                    'game_name': game_info['name'],  # 게임 이름
                    'game_genres': game_info['genres'],  # 게임 장르
                    'game_image': game_info['header_image'],  # 게임 이미지
                    'game_release_date': game_info['release_date'],  # 출시일
                    'game_price': game_info['price'],  # 가격
                    'review_text': review['review_text'],  # 리뷰 내용
                    'review_score': review['review_score'],  # 리뷰 평가
                    'language': review['language'],  # 리뷰 언어
                    'author': author_nickname,  # 작성자 닉네임
                    'playtime_at_review': review['playtime_at_review'],  # 작성 당시 플레이 시간
                    'timestamp_created': review['timestamp_created']  # 리뷰 작성 시간
                })

    # 모든 데이터를 DataFrame으로 변환
    df = pd.DataFrame(all_data)
    return df

# 게임 데이터 및 리뷰 가져오기
df_game_data = get_game_data_with_reviews(app_id, num_reviews=50)  # 각 게임당 50개의 리뷰 가져오기

# CSV 파일로 저장
df_game_data.to_csv('3d.csv', index=False)


In [3]:
Example=pd.read_csv('3d.csv')
Example

,app_id,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,1049590,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,The game is a commercial failure. Wouldn't sur...,False,english,76561198110879950,2371,1708398574
1,1049590,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,one thing give a slight nerf on op bianca?\n,True,english,76561198208821009,279,1638476898
2,1049590,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,Fun af.\nEdit: About 1500 hours later game is ...,True,english,76561198200647247,3700,1635721171
3,1049590,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,actually addicting,True,english,76561198276917081,417,1625543925
4,1049590,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,fun game challenging and overall good communit...,True,english,76561198182529301,24060,1620058458
...,...,...,...,...,...,...,...,...,...,...,...,...
414,872410,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,"15 Jan, 2020","₩ 38,880",great game,True,english,76561198410034150,25753,1703047548
415,872410,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,"15 Jan, 2020","₩ 38,880",Great Simulation Game,True,english,76561198124592679,11593,1702722343
416,872410,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,"15 Jan, 2020","₩ 38,880",This game is not well-designed at all. The fir...,False,english,76561199548905738,9921,1701888717
417,872410,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,"15 Jan, 2020","₩ 38,880",最高的定价最烂的内容，编辑器都要分批购买，第四wave我买下来后甚至download不了，一...,False,english,76561198346517187,4946,1701826055


In [4]:
Example['app_id'].value_counts()

app_id
1245620    50
2358720    50
1364780    50
2183900    50
2322010    50
2021880    50
1462040    50
872410     50
1049590    19
Name: count, dtype: int64

In [5]:
Example1= Example.copy()
Example1

,app_id,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,1049590,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,The game is a commercial failure. Wouldn't sur...,False,english,76561198110879950,2371,1708398574
1,1049590,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,one thing give a slight nerf on op bianca?\n,True,english,76561198208821009,279,1638476898
2,1049590,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,Fun af.\nEdit: About 1500 hours later game is ...,True,english,76561198200647247,3700,1635721171
3,1049590,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,actually addicting,True,english,76561198276917081,417,1625543925
4,1049590,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,fun game challenging and overall good communit...,True,english,76561198182529301,24060,1620058458
...,...,...,...,...,...,...,...,...,...,...,...,...
414,872410,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,"15 Jan, 2020","₩ 38,880",great game,True,english,76561198410034150,25753,1703047548
415,872410,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,"15 Jan, 2020","₩ 38,880",Great Simulation Game,True,english,76561198124592679,11593,1702722343
416,872410,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,"15 Jan, 2020","₩ 38,880",This game is not well-designed at all. The fir...,False,english,76561199548905738,9921,1701888717
417,872410,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,"15 Jan, 2020","₩ 38,880",最高的定价最烂的内容，编辑器都要分批购买，第四wave我买下来后甚至download不了，一...,False,english,76561198346517187,4946,1701826055


In [6]:
Example1=Example1.drop(['app_id', ], axis=1) # app_id 삭제
Example1 = Example1[Example1['language'] == 'english'] # english 
Example1 = Example1.reset_index(drop=True) # 재정렬
Example1 

,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,The game is a commercial failure. Wouldn't sur...,False,english,76561198110879950,2371,1708398574
1,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,one thing give a slight nerf on op bianca?\n,True,english,76561198208821009,279,1638476898
2,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,Fun af.\nEdit: About 1500 hours later game is ...,True,english,76561198200647247,3700,1635721171
3,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,actually addicting,True,english,76561198276917081,417,1625543925
4,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,"19 Jul, 2023",가격 정보 없음,fun game challenging and overall good communit...,True,english,76561198182529301,24060,1620058458
...,...,...,...,...,...,...,...,...,...,...,...
414,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,"15 Jan, 2020","₩ 38,880",great game,True,english,76561198410034150,25753,1703047548
415,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,"15 Jan, 2020","₩ 38,880",Great Simulation Game,True,english,76561198124592679,11593,1702722343
416,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,"15 Jan, 2020","₩ 38,880",This game is not well-designed at all. The fir...,False,english,76561199548905738,9921,1701888717
417,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,"15 Jan, 2020","₩ 38,880",最高的定价最烂的内容，编辑器都要分批购买，第四wave我买下来后甚至download不了，一...,False,english,76561198346517187,4946,1701826055


In [7]:
# Example1 전처리
Example1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   game_name           419 non-null    object
 1   game_genres         419 non-null    object
 2   game_image          419 non-null    object
 3   game_release_date   419 non-null    object
 4   game_price          419 non-null    object
 5   review_text         418 non-null    object
 6   review_score        419 non-null    bool  
 7   language            419 non-null    object
 8   author              419 non-null    int64 
 9   playtime_at_review  419 non-null    int64 
 10  timestamp_created   419 non-null    int64 
dtypes: bool(1), int64(3), object(7)
memory usage: 33.3+ KB


In [8]:
Example1['game_name'].isnull()
Example1['game_name'].value_counts() # 개수확인  총합 .sum()
Example1['game_name'].isnull() # null 확인 총합 .sum()

0      False
1      False
2      False
3      False
4      False
       ...  
414    False
415    False
416    False
417    False
418    False
Name: game_name, Length: 419, dtype: bool

In [9]:
Example1['game_genres'].value_counts().sum()
Example1['game_genres'].isnull().sum()

0

In [10]:
Example1['game_image'].value_counts().sum()
Example1['game_image'].isnull().sum()



0

In [11]:
Example1['language'].value_counts().sum()
Example1['language'].isnull().sum()

0

In [12]:
Example1['game_release_date'].value_counts().sum()
Example1['game_release_date'].isnull().sum()
Example1['game_release_date'].value_counts()


# 이탈리아어 월 이름을 영어로 변환하는 사전
month_map = {
    'gen.': 'Jan',
    'feb.': 'Feb',
    'mar.': 'Mar',
    'apr.': 'Apr',
    'mag.': 'May',
    'giu.': 'Jun',
    'lug.': 'Jul',
    'ago.': 'Aug',
    'set.': 'Sep',
    'ott.': 'Oct',
    'nov.': 'Nov',
    'dic.': 'Dec'
}

# 날짜 변환 함수 정의
def parse_date(date_str):
    # 이탈리아어 월 이름을 영어로 변환
    for italian_month, english_month in month_map.items():
        if italian_month in date_str:
            date_str = date_str.replace(italian_month, english_month)

    # 여러 가지 날짜 형식 처리
    try:
        # 'Sep 19, 2024' 형식
        return pd.to_datetime(date_str, format='%b %d, %Y')
    except ValueError:
        pass
    try:
        # '19 Sep, 2024' 형식
        return pd.to_datetime(date_str, format='%d %b, %Y')
    except ValueError:
        pass
    try:
        # '9/set./2024' 형식 (이탈리아어 월 변환 후 처리)
        return pd.to_datetime(date_str, format='%d/%b/%Y')
    except ValueError:
        pass
    
    return pd.NaT  # 변환 실패 시 NaT 반환

# 데이터프레임의 날짜 열 변환
Example1['game_release_date'] = Example1['game_release_date'].apply(parse_date)
Example1['game_release_date'].isnull().sum()
Example1['game_release_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 419 entries, 0 to 418
Series name: game_release_date
Non-Null Count  Dtype         
--------------  -----         
369 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 3.4 KB


In [13]:
Example1['review_score']
Example1['review_score'].isnull().sum()
Example1['review_score'].value_counts()
Example1['review_score'].value_counts().sum()

419

In [14]:
Example1['review_text'].value_counts().sum()
Example1['review_text'].value_counts()
Example1['review_text'].isnull().sum()
Example1['review_text'].isnull()
Example1 = Example1.dropna(subset=['review_text'], axis=0)

In [15]:
# game_price
Example1['game_price'].value_counts().sum()

418

In [16]:
Example1['game_price'].isnull().sum()


0

In [17]:
Example1['game_price'].value_counts()

game_price
₩ 45,360     50
₩ 64,800     50
₩ 34,900     50
3199 руб.    50
₩ 59,900     50
¥ 223.00     50
₩ 38,880     50
$59.99       49
가격 정보 없음     19
Name: count, dtype: int64

In [18]:
# 특정 값 변경
Example1['game_price'] = Example1['game_price'].replace('44,99€', '44.99€') # 

Example1['game_price'] = Example1['game_price'].replace('R$ 199,90', 'R$ 199.90')

C:\Users\user\AppData\Local\Temp\ipykernel_19500\107871597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['game_price'] = Example1['game_price'].replace('44,99€', '44.99€') #
C:\Users\user\AppData\Local\Temp\ipykernel_19500\107871597.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['game_price'] = Example1['game_price'].replace('R$ 199,90', 'R$ 199.90')


In [19]:
# 특정 값 변경
Example1['game_price'] = Example1['game_price'].replace('44,99€', '44.99€') 

Example1['game_price'] = Example1['game_price'].replace('R$ 199,90', 'R$ 199.90')

C:\Users\user\AppData\Local\Temp\ipykernel_19500\4294488823.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['game_price'] = Example1['game_price'].replace('44,99€', '44.99€')
C:\Users\user\AppData\Local\Temp\ipykernel_19500\4294488823.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['game_price'] = Example1['game_price'].replace('R$ 199,90', 'R$ 199.90')


In [20]:
import re

# 통화 기호에 따라 숫자를 변환하는 함수
def extract_numbers(price_str):
    if isinstance(price_str, str):  # price_str이 문자열인지 확인
        # 통화 기호에 따라 환율을 설정
        if '₩' in price_str:
            multiplier = 1  # 원화: 그대로 사용
        elif 'CDN$' in price_str:
            multiplier = 950  # 캐나다 달러 기준, 예: 1 CAD = 950 KRW
        elif '$' in price_str:
            multiplier = 1300  # 미국 달러 기준, 예: 1 USD = 1300 KRW
        elif '€' in price_str:
            multiplier = 1400  # 유로 기준, 예: 1 EUR = 1400 KRW
        elif '¥' in price_str:
            multiplier = 10  # 엔화 기준, 예: 1 JPY = 10 KRW
        elif 'R$' in price_str:
            multiplier = 300  # 브라질 헤알 기준, 예: 1 BRL = 300 KRW
        else:
            multiplier = 1  # 통화 기호가 없는 경우 그대로 사용

        # 소수점이 포함된 숫자만 추출 (쉼표와 공백 제거)
        numbers = re.sub(r'[^\d.]', '', price_str)
        
        # 추출한 숫자 문자열을 소수로 변환한 후 환율 적용
        return float(numbers) * multiplier if numbers else 0  # 숫자에 통화 단위 곱하기
    return 0  # 문자열이 아닐 경우 0 반환

# 'game_price' 열 변환
Example1['game_price'] = Example1['game_price'].replace('가격 정보 없음', '0')  # 가격 정보가 없는 경우 0으로 대체
Example1['game_price'] = Example1['game_price'].apply(extract_numbers)  # 각 가격 문자열에 대해 함수 적용

C:\Users\user\AppData\Local\Temp\ipykernel_19500\1318368436.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['game_price'] = Example1['game_price'].replace('가격 정보 없음', '0')  # 가격 정보가 없는 경우 0으로 대체
C:\Users\user\AppData\Local\Temp\ipykernel_19500\1318368436.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['game_price'] = Example1['game_price'].apply(extract_numbers)  # 각 가격 문자열에 대해 함수 적용


In [21]:
Example1['game_price'].value_counts()


game_price
45360.0    50
64800.0    50
34900.0    50
3199.0     50
59900.0    50
2230.0     50
38880.0    50
77987.0    49
0.0        19
Name: count, dtype: int64

In [22]:
Example1['game_price'].isnull().sum()


0

In [23]:
Example1['game_price'].astype(int)

0          0
1          0
2          0
3          0
4          0
       ...  
414    38880
415    38880
416    38880
417    38880
418    38880
Name: game_price, Length: 418, dtype: int32

In [24]:
Example1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 0 to 418
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   game_name           418 non-null    object        
 1   game_genres         418 non-null    object        
 2   game_image          418 non-null    object        
 3   game_release_date   368 non-null    datetime64[ns]
 4   game_price          418 non-null    float64       
 5   review_text         418 non-null    object        
 6   review_score        418 non-null    bool          
 7   language            418 non-null    object        
 8   author              418 non-null    int64         
 9   playtime_at_review  418 non-null    int64         
 10  timestamp_created   418 non-null    int64         
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 36.3+ KB


In [25]:
# review_text 영어만 인식한것만
def is_english(text):
    # 문자열이 None이 아닐 때 영어 문자가 포함되어 있는지 확인
    if isinstance(text, str):
        return bool(re.match(r'^[\x00-\x7F]*$', text))
    return False

# 영어가 아닌 행의 인덱스 찾기 (조건 반전)
indices_to_drop = Example1[~Example1['review_text'].apply(is_english)].index

# 기존 데이터프레임에서 영어가 아닌 행 삭제
Example1.drop(indices_to_drop, inplace=True)

# 결과 출력
Example1

C:\Users\user\AppData\Local\Temp\ipykernel_19500\1213455055.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1.drop(indices_to_drop, inplace=True)


,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,2023-07-19,0.0,The game is a commercial failure. Wouldn't sur...,False,english,76561198110879950,2371,1708398574
1,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,2023-07-19,0.0,one thing give a slight nerf on op bianca?\n,True,english,76561198208821009,279,1638476898
2,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,2023-07-19,0.0,Fun af.\nEdit: About 1500 hours later game is ...,True,english,76561198200647247,3700,1635721171
3,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,2023-07-19,0.0,actually addicting,True,english,76561198276917081,417,1625543925
4,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,2023-07-19,0.0,fun game challenging and overall good communit...,True,english,76561198182529301,24060,1620058458
...,...,...,...,...,...,...,...,...,...,...,...
413,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,2020-01-15,38880.0,I have not played the game for many years. 14 ...,True,english,76561198031146306,830,1703636670
414,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,2020-01-15,38880.0,great game,True,english,76561198410034150,25753,1703047548
415,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,2020-01-15,38880.0,Great Simulation Game,True,english,76561198124592679,11593,1702722343
416,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,2020-01-15,38880.0,This game is not well-designed at all. The fir...,False,english,76561199548905738,9921,1701888717


In [26]:
#댓글 날짜, 닉네임, 사용자 플레이 시간 전처리

# 댓글 날짜 
Example1['playtime_at_review'].value_counts()
Example1['playtime_at_review'].isnull().sum() # 417
Example1['playtime_at_review'].value_counts().sum() # 6581
Example1['playtime_at_review']=Example1['playtime_at_review']/60
Example1['playtime_at_review']=Example1['playtime_at_review'].fillna(0)
Example1['playtime_at_review'].isnull().sum()
Example1['playtime_at_review'].astype(int)


# 사용자 플레이 시간 
Example1['timestamp_created'].isnull().sum() # 0
Example1['timestamp_created'].value_counts().sum() # 6581
#  Unix 타임스탬프
import pandas as pd

Example1['timestamp_created']
Example1['timestamp_created']=pd.to_datetime(Example1['timestamp_created'], unit='s')




C:\Users\user\AppData\Local\Temp\ipykernel_19500\2510639514.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['playtime_at_review']=Example1['playtime_at_review']/60
C:\Users\user\AppData\Local\Temp\ipykernel_19500\2510639514.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['playtime_at_review']=Example1['playtime_at_review'].fillna(0)
C:\Users\user\AppData\Local\Temp\ipykernel_19500\2510639514.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [27]:
Example1['game_name']

0                         Eternal Return
1                         Eternal Return
2                         Eternal Return
3                         Eternal Return
4                         Eternal Return
                     ...                
413    ROMANCE OF THE THREE KINGDOMS XIV
414    ROMANCE OF THE THREE KINGDOMS XIV
415    ROMANCE OF THE THREE KINGDOMS XIV
416    ROMANCE OF THE THREE KINGDOMS XIV
418    ROMANCE OF THE THREE KINGDOMS XIV
Name: game_name, Length: 402, dtype: object

In [28]:
Example1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 402 entries, 0 to 418
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   game_name           402 non-null    object        
 1   game_genres         402 non-null    object        
 2   game_image          402 non-null    object        
 3   game_release_date   353 non-null    datetime64[ns]
 4   game_price          402 non-null    float64       
 5   review_text         402 non-null    object        
 6   review_score        402 non-null    bool          
 7   language            402 non-null    object        
 8   author              402 non-null    int64         
 9   playtime_at_review  402 non-null    float64       
 10  timestamp_created   402 non-null    datetime64[ns]
dtypes: bool(1), datetime64[ns](2), float64(2), int64(1), object(5)
memory usage: 34.9+ KB


In [29]:
Example1.to_csv('3d.csv')

In [30]:
ex=pd.read_csv('3d.csv')
ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          402 non-null    int64  
 1   game_name           402 non-null    object 
 2   game_genres         402 non-null    object 
 3   game_image          402 non-null    object 
 4   game_release_date   353 non-null    object 
 5   game_price          402 non-null    float64
 6   review_text         402 non-null    object 
 7   review_score        402 non-null    bool   
 8   language            402 non-null    object 
 9   author              402 non-null    int64  
 10  playtime_at_review  402 non-null    float64
 11  timestamp_created   402 non-null    object 
dtypes: bool(1), float64(2), int64(2), object(7)
memory usage: 35.1+ KB


In [31]:
ex

,Unnamed: 0,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,0,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,2023-07-19,0.0,The game is a commercial failure. Wouldn't sur...,False,english,76561198110879950,39.516667,2024-02-20 03:09:34
1,1,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,2023-07-19,0.0,one thing give a slight nerf on op bianca?\n,True,english,76561198208821009,4.650000,2021-12-02 20:28:18
2,2,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,2023-07-19,0.0,Fun af.\nEdit: About 1500 hours later game is ...,True,english,76561198200647247,61.666667,2021-10-31 22:59:31
3,3,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,2023-07-19,0.0,actually addicting,True,english,76561198276917081,6.950000,2021-07-06 03:58:45
4,4,Eternal Return,"Indie, Strategy, Free To Play",https://shared.akamai.steamstatic.com/store_it...,2023-07-19,0.0,fun game challenging and overall good communit...,True,english,76561198182529301,401.000000,2021-05-03 16:14:18
...,...,...,...,...,...,...,...,...,...,...,...,...
397,413,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,2020-01-15,38880.0,I have not played the game for many years. 14 ...,True,english,76561198031146306,13.833333,2023-12-27 00:24:30
398,414,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,2020-01-15,38880.0,great game,True,english,76561198410034150,429.216667,2023-12-20 04:45:48
399,415,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,2020-01-15,38880.0,Great Simulation Game,True,english,76561198124592679,193.216667,2023-12-16 10:25:43
400,416,ROMANCE OF THE THREE KINGDOMS XIV,Simulation,https://shared.akamai.steamstatic.com/store_it...,2020-01-15,38880.0,This game is not well-designed at all. The fir...,False,english,76561199548905738,165.350000,2023-12-06 18:51:57
